In [4]:
import os
import cv2
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog
import matplotlib.pyplot as plt

# 📌 Constants
IMG_SIZE = (64, 64)  # Resize all images to 64x64
DATA_PATH = '../data/'

def load_images(folder):
    images = []
    for file in os.listdir(folder):
        if file.lower().endswith(('.jpg', '.png', '.jpeg')):
            path = os.path.join(folder, file)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMG_SIZE)
            images.append(img)
    return images

# 🔹 Load images
car_imgs = load_images(os.path.join(DATA_PATH, 'vehicle'))
noncar_imgs = load_images(os.path.join(DATA_PATH, 'non-vehicle'))

# 🔹 Labels
X = car_imgs + noncar_imgs
y = [1]*len(car_imgs) + [0]*len(noncar_imgs)

print(f"Loaded {len(X)} images. Cars: {len(car_imgs)}, Non-Cars: {len(noncar_imgs)}")

# 🔹 HOG Feature Extraction
def extract_hog_features(images):
    features = []
    for img in images:
        hog_feature = hog(img, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features.append(hog_feature)
    return features

X_features = extract_hog_features(X)

# 🔹 Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.3, random_state=42)

# 🔹 Train SVM
model = LinearSVC()
model.fit(X_train, y_train)

# 🔹 Predict
y_pred = model.predict(X_test)

# 🔹 Evaluation
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Loaded 61 images. Cars: 30, Non-Cars: 31
✅ Accuracy: 0.7894736842105263

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80        10
           1       0.78      0.78      0.78         9

    accuracy                           0.79        19
   macro avg       0.79      0.79      0.79        19
weighted avg       0.79      0.79      0.79        19

